<a href="https://colab.research.google.com/github/radhakrishnan-omotec/avm-repository/blob/master/Sourish_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # used for visualization purpose
import seaborn as sb
import tensorflow as tf
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os
import pathlib

In [ ]:
data_dir=os.path.join("/content/gdrive/MyDrive/Sourish_Project_Dataset")
data_dir=pathlib.Path(data_dir).with_suffix("")

In [ ]:
x=os.scandir(data_dir)
for i in x:
  print(i.name)

RICE-BACTERIAL BLIGHT
RICE-BROWN SPOT
POTATO-EARLY BLIGHT
POTATO-LATE BLIGHT
CORN-GREY LEAF SPOT
TOMATO-EARLY BLIGHT
WHEAT-BROWN RUST
WHEAT-YELLOW RUST


# Pre-Processing

In [ ]:
batch_size=32
img_height=200
img_width=200

In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset="training",seed=123,image_size=(img_height,img_width),batch_size=batch_size)

Found 5619 files belonging to 8 classes.
Using 4496 files for training.


In [ ]:
validation_ds=tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset="validation",seed=123,image_size=(img_height,img_width),batch_size=batch_size)

Found 5619 files belonging to 8 classes.
Using 1123 files for validation.


In [ ]:
class_names=train_ds.class_names
print(class_names)

['CORN-GREY LEAF SPOT', 'POTATO-EARLY BLIGHT', 'POTATO-LATE BLIGHT', 'RICE-BACTERIAL BLIGHT', 'RICE-BROWN SPOT', 'TOMATO-EARLY BLIGHT', 'WHEAT-BROWN RUST', 'WHEAT-YELLOW RUST']


# Pre-processing for all the images

In [ ]:
normalization_layer= layers.Rescaling(1./255)

In [ ]:
normalized_ds= train_ds.map(lambda x,y:(normalization_layer(x),y))
image_batch, labels_batch=next(iter(normalized_ds))
first_image=image_batch[0]
print(np.min(first_image),np.max(first_image))

0.0035104451 0.99242365


# CNN Model Structure

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# CNN Model Design
num_classes=len(class_names)
model=Sequential([
    layers.Rescaling(1./255,input_shape=(img_height,img_width,3)),
    layers.Conv2D(16,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(32,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(64,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(128,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation="relu"),
    layers.Dense(num_classes)])

In [ ]:
model.compile(optimizer="adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy","mse"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 200, 200, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 200, 200, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 16)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 100, 100, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 50, 50, 32)        0         
 g2D)                                                   

In [ ]:
#epochs=100
#history=model.fit(train_ds,validation_data=validation_ds,epochs=epochs)

Epoch 1/100
141/141 [==============================] - 1222s 9s/step - loss: 1.2176 - accuracy: 0.5269 - mse: 27.9496 - val_loss: 1.1217 - val_accuracy: 0.5343 - val_mse: 20.9570
Epoch 2/100
141/141 [==============================] - 27s 182ms/step - loss: 0.5786 - accuracy: 0.7811 - mse: 39.2034 - val_loss: 0.6254 - val_accuracy: 0.7970 - val_mse: 24.5111
Epoch 3/100
141/141 [==============================] - 27s 185ms/step - loss: 0.3517 - accuracy: 0.8719 - mse: 47.9116 - val_loss: 0.4680 - val_accuracy: 0.8593 - val_mse: 29.2252
Epoch 4/100
141/141 [==============================] - 27s 185ms/step - loss: 0.2692 - accuracy: 0.9061 - mse: 56.9599 - val_loss: 0.4104 - val_accuracy: 0.8718 - val_mse: 35.3721
Epoch 5/100
141/141 [==============================] - 31s 216ms/step - loss: 0.2173 - accuracy: 0.9230 - mse: 69.4561 - val_loss: 0.4329 - val_accuracy: 0.8557 - val_mse: 34.8783
Epoch 6/100
141/141 [==============================] - 27s 183ms/step - loss: 0.1722 - accuracy: 0.93

In [ ]:
model.save("Crop_Disease_Classification_Model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# To check the accuracy of separate classses separately

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
import os
import pathlib
from tensorflow.keras import layers

# Define your parameters
batch_size = 32
img_height = 200
img_width = 200
data_dir = os.path.join("/content/gdrive/MyDrive/Sourish_Project_Dataset")
data_dir = pathlib.Path(data_dir).with_suffix("")

# Load the dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Normalize the dataset
normalization_layer = layers.Rescaling(1./255)
normalized_validation_ds = validation_ds.map(lambda x, y: (normalization_layer(x), y))

# Check the normalization
image_batch, labels_batch = next(iter(normalized_validation_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

# Assuming you have your model loaded
# model = ... (load your model here)

# Initialize lists to store true labels and predictions
y_true = []
y_pred = []

# Predict on validation dataset
for images, labels in normalized_validation_ds:
    predictions = model.predict(images)
    y_pred.extend(np.argmax(predictions, axis=1))
    y_true.extend(labels.numpy())

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred)
print(f'Overall Accuracy: {overall_accuracy}')

# Get the class names
class_names = validation_ds.class_names

# Calculate per-class accuracy
classes = np.unique(y_true)
class_accuracies = {}

for cls in classes:
    cls_idx = np.where(y_true == cls)
    cls_accuracy = accuracy_score(y_true[cls_idx], y_pred[cls_idx])
    class_accuracies[class_names[cls]] = cls_accuracy
    print(f'Accuracy for class {class_names[cls]}: {cls_accuracy}')

print('Per-class Accuracies:', class_accuracies)


Found 5619 files belonging to 8 classes.
Using 4496 files for training.
Found 5619 files belonging to 8 classes.
Using 1123 files for validation.
0.0 1.0
1/1 [==============================] - 0s 85ms/step
Overall Accuracy: 0.0819234194122885
Accuracy for class CORN-GREY LEAF SPOT: 0.0
Accuracy for class POTATO-EARLY BLIGHT: 0.0
Accuracy for class POTATO-LATE BLIGHT: 0.0
Accuracy for class RICE-BACTERIAL BLIGHT: 0.0
Accuracy for class RICE-BROWN SPOT: 0.0
Accuracy for class TOMATO-EARLY BLIGHT: 1.0
Accuracy for class WHEAT-BROWN RUST: 0.0
Accuracy for class WHEAT-YELLOW RUST: 0.0
Per-class Accuracies: {'CORN-GREY LEAF SPOT': 0.0, 'POTATO-EARLY BLIGHT': 0.0, 'POTATO-LATE BLIGHT': 0.0, 'RICE-BACTERIAL BLIGHT': 0.0, 'RICE-BROWN SPOT': 0.0, 'TOMATO-EARLY BLIGHT': 1.0, 'WHEAT-BROWN RUST': 0.0, 'WHEAT-YELLOW RUST': 0.0}


# To Test Model